# Obsah

> ## Úvod

> ## Knižnice

> ## Dataset

> ## Chýbajúce hodnoty

>> ### Zjednotenie formátov dát

>> ### Odvodenie chýbajúcich hodnôt po riadkoch

>> ### Doplnenie chýbajúcich hodnôt po stĺpcoch

> ## Vychýlené hodnoty

> ## Normalizácia údajov

> ## Transformácia numerických atribútov na kategorické

> ## Transformacia kategorických údajov na numerické

> ## Vytváranie atribútov kombinovanim

> ## Zdroje

## Doležité parametre
DecisionTreeClassifier(
criterion

max_depth=None

min_samples_leaf - počet pozorovani vo vyslednej podmnozine

class_weight=None

# Úvod
Na základe identifikovaných problémov v dátach a návrhu ich riešenia v predchádzajúcej fáze treba zrealizovať predspracovanie. Výsledkom by mala byť upravená dátová sada (vo formáte ​csv​) vo vhodnom tvare pre zvolený algoritmus strojového učenia (v našom prípade rozhodovacie stromy​). Zároveň, keďže predspracovaním sa mohol zmeniť tvar a charakteristiky dát (počet atribútov, distribúcie hodnôt a pod.), treba znovu zrealizovať podstatné časti prieskumnej analýzy, pričom v tejto fáze je potrebné zamerať sa aj na vzťahy medzi jednotlivými atribútmi.

# Knižnice

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn import preprocessing

#regular expression
import re

#Nastavenie rozmerov zobrazovaných grafov
plt.rcParams['figure.figsize'] = 9, 6
from IPython.display import Image

# Dataset

In [101]:
file_path = "data/zadanie4/train.csv"
# Načítame si súbor s datasetom so separátom ',' a ako index nastavime prvý stĺpec
train_data = pd.read_csv(file_path, sep=',', index_col=0)
# Pomenujeme si index
train_data.index.name = 'id'
#train_data.info()

In [102]:
file_path = "data/zadanie4/test.csv"
# Načítame si súbor s datasetom so separátom ',' a ako index nastavime prvý stĺpec
test_data = pd.read_csv(file_path, sep=',', index_col=0)
# Pomenujeme si index
test_data.index.name = 'id'
#test_data.info()

In [103]:
#Aby sme mohli pracovat 
#data = train_data
data = test_data

In [104]:
def fixWierdCharsInString(string):
    #ocisti string od bielych znakov a regex: nahrad znaky [ -], za _
    return re.sub('[ -]','_', str.strip(string))

In [105]:
#Zjednotenie názvov stĺpcov
data = data.rename(columns=fixWierdCharsInString)

In [106]:
data = data.rename(fixWierdCharsInString, axis='columns')
data.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'class', 'fnlwgt', 'education', 'education_num', 'relationship',
       'capital_gain', 'capital_loss', 'hours_per_week', 'date_of_birth',
       'personal_info'],
      dtype='object')

# Zjednotenie formátov dát

In [107]:
#help(data.select_dtypes)
#Selektneme si vsetky atribúty, ktoré majú nejasný dátový
for col_name in data.select_dtypes('object'):
    #Spočítame si pre každý "kategorický" atribút počet jeho jedinečných hodnôt
    uniq_values_count = len(data[col_name].unique())
    # vypíšeme si počty
    print(col_name,":",uniq_values_count) 

sex : 3
on_thyroxine : 6
query_on_thyroxine : 2
on_antithyroid_medication : 2
sick : 2
pregnant : 2
thyroid_surgery : 2
I131_treatment : 2
query_hypothyroid : 2
query_hyperthyroid : 2
lithium : 2
goitre : 2
tumor : 2
hypopituitary : 1
psych : 2
TSH_measured : 2
T3_measured : 2
TT4_measured : 2
T4U_measured : 2
FTI_measured : 2
FTI : 162
TBG_measured : 1
TBG : 1
referral_source : 5
class : 972
education : 16
relationship : 9
date_of_birth : 867
personal_info : 338


Pre všetky vyššie atribúty nastavíme správny formát dát a prekonvertujeme na číselnú reprezentáciu, kvôli neskoršej klasifikácií.

In [108]:
#help(data.to_numeric)
#If ‘coerce’, then invalid parsing will be set as NaN
#FTI prekonvertujeme na čísla
data['FTI'] = pd.to_numeric(data['FTI'], errors='coerce')

Zmena hodnôt **M -> Male** a **F -> Female**, **? -> NaN**. 


PJ NEROZUMIEM: Túto úpravu robíme kvôli tomu, aby sme pri upravovaní hodnôt F, f, T, t ... mohli upraviť tieto hodnoty globálne.

In [109]:
data['sex'].value_counts()

F    650
M    282
?     40
Name: sex, dtype: int64

In [110]:
#Nastavíme pohlavie na jeden formát
data['sex'].replace(['F', 'M', '?'], [0, 1, np.NAN], inplace=True)
data['sex'].value_counts()

0.0    650
1.0    282
Name: sex, dtype: int64

In [111]:
data.rename(columns={'sex':'sex_male'})

,age,sex_male,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,class,fnlwgt,education,education_num,relationship,capital_gain,capital_loss,hours_per_week,date_of_birth,personal_info
id,,,,,,,,,,,,,,,,,,,,,
0,68,1.0,f,f,f,f,f,f,f,f,...,negative.|1231,96245,HS-grad,9,Husband,0.0,0,99,1949-07-30,Sales|United-States\nMarried-civ-spouse -- Sel...
1,42,0.0,f,f,f,f,f,f,f,f,...,negative.|329,191177,HS-grad,9,Not-in-family,NaN,0,55,1975-10-31,Exec-managerial|United-States\nNever-married -...
2,27,0.0,f,f,f,f,f,f,f,f,...,negative.|2518,48585,Some-college,1000,Wife,0.0,0,4,1990-10-29 00:00:00,Prof-specialty|United-States\nMarried-civ-spou...
3,32,0.0,FALSE,f,f,f,f,f,f,f,...,negative.|821,251526,Some-college,10,Wife,0.0,0,20,1985-10-28,Tech-support|United-States\nMarried-civ-spouse...
4,23,0.0,f,f,f,f,f,f,f,f,...,negative.|951,175614,10th,6,Unmarried,0.0,0,40,??,Other-service|United-States\nNever-married -- ...
5,29,0.0,t,f,f,f,f,f,f,f,...,underreplacement.|677,122857,HS-grad,9,Not-in-family,0.0,0,40,1988-10-12,Priv-house-serv|?\nNever-married -- Private|As...
6,50,0.0,t,f,f,f,f,f,f,f,...,replacement therapy.|1863,340217,Some-college,10,Not-in-family,0.0,0,20,1967/06/01,Sales|United-States\nNever-married -- Private|...
7,45,1.0,f,f,f,f,f,f,f,f,...,negative.|1967,149771,HS-grad,9,Not-in-family,3325.0,0,40,NaN,Craft-repair|United-States\nDivorced -- Privat...
8,68,0.0,f,f,f,f,f,f,f,f,...,negative.|1372,223921,12th,8,Own-child,0.0,0,40,1949-12-10 00 00 00,Machine-op-inspct|United-States\nNever-married...


Zmena hodnôt **f, F, FALSE -> False** a **t, T, TRUE -> True**

In [112]:
data.replace(['f', 'F', 'FALSE', 't', 'T', 'TRUE'], [False, False, False, True, True, True], inplace=True)

Ujednotenie názvov v stĺpci **relationship**. 
Own-child -> Own_child
Not-in-family -> Not_in_family
Other-relative -> Other_relative

In [113]:
data['relationship'].value_counts()

 Not-in-family     274
 Husband           186
 Unmarried         175
 Own-child         152
 Wife               98
 Other-relative     37
 Not_in_family      24
 Own_child          20
 Other_relative      6
Name: relationship, dtype: int64

In [114]:
data['relationship'] = data['relationship'].apply(fixWierdCharsInString)

In [115]:
data['relationship'].value_counts()

Not_in_family     298
Husband           186
Unmarried         175
Own_child         172
Wife               98
Other_relative     43
Name: relationship, dtype: int64

In [116]:
data['relationship'] = data['relationship'].apply(str.strip)

'''data['relationship'] = data['relationship'].map({
    'Own-child': 'Own_child', 
    'Not-in-family': 'Not_in_family', 
    'Other-relative': 'Other_relative',
    'Unmarried': 'Unmarried',
    'Wife': 'Wife',
    'Husband': 'Husband'
})'''

data['relationship'].replace(['Own-child', 'Not-in-family', 'Other-relative'], ['Own_child', 'Not_in_family', 'Other_relative'], inplace=True)

Ujednotenie formátu atribútu **date_of_birth**

In [117]:
def convertToDateType(string):
    if pd.isna(string) or string == '??':
        return pd.NaT
    #Nastavime si datum na 10 znakov
    if isinstance(string, str):
        string = string.strip()[:10]
        if len(string) < 10:
            if(re.match("[01][0-8]",string[:2])):
                #print(string)
                string = '20'+ string
            elif(re.match("[2-9][0-9]",string[:2])): 
                string = '19'+ string
                #print(string)
    try: return pd.to_datetime(string, errors='raise')
    except ValueError:
            #print(string)
            return pd.NaT

In [118]:
data['date_of_birth'] = pd.to_datetime(data['date_of_birth'], errors='coerce')
data['date_of_birth'].describe()

count                     811
unique                    790
top       1969-10-26 00:00:00
freq                        2
first     1924-10-04 00:00:00
last      2066-10-15 00:00:00
Name: date_of_birth, dtype: object

In [119]:
data['date_of_birth'] = data['date_of_birth'].apply(convertToDateType)
data['date_of_birth'].describe()

count                     811
unique                    790
top       1969-10-26 00:00:00
freq                        2
first     1924-10-04 00:00:00
last      2066-10-15 00:00:00
Name: date_of_birth, dtype: object

Rozdelenie stĺpca **personal_info** do viacerých stĺpcov

In [120]:
df_personal_data = pd.DataFrame(data['personal_info'].str.replace('\n','|').str.replace('--','|').str.split('|', expand=False).tolist(), columns=['job','state', 'parents_definition', 'segment', 'race'])

Atribút je teraz rozdelený do viacerých stĺpcov **job, state, parents_definition, segment, race**, ale je potrebné v nich urobiť nejaké úpravy. 

In [121]:
df_personal_data.head()

,job,state,parents_definition,segment,race
0,Sales,United-States,Married-civ-spouse,Self-emp-inc,White
1,Exec-managerial,United-States,Never-married,Private,White
2,Prof-specialty,United-States,Married-civ-spouse,Self-emp-not-inc,White
3,Tech-support,United-States,Married-civ-spouse,Private,White
4,Other-service,United-States,Never-married,Private,White


In [122]:
df_personal_data['job'].value_counts()

Other-service        150
Adm-clerical         146
Sales                130
Prof-specialty       121
Exec-managerial      112
?                     76
Craft-repair          69
Machine-op-inspct     58
Tech-support          32
Transport-moving      25
Handlers-cleaners     17
Priv-house-serv       13
Farming-fishing       13
Protective-serv       10
Name: job, dtype: int64

In [123]:
for col in df_personal_data.columns:
    df_personal_data[col] = df_personal_data[col].str.replace('-','_')

df_personal_data.head()

,job,state,parents_definition,segment,race
0,Sales,United_States,Married_civ_spouse,Self_emp_inc,White
1,Exec_managerial,United_States,Never_married,Private,White
2,Prof_specialty,United_States,Married_civ_spouse,Self_emp_not_inc,White
3,Tech_support,United_States,Married_civ_spouse,Private,White
4,Other_service,United_States,Never_married,Private,White


Teraz môžeme vytvorené stĺpce z atribútu **personal_info** pripojiť k celému datasetu a vymazať atribút **personal_info**.

In [124]:
data = data.drop(['personal_info'], axis=1) # vymazanie atribútu personal_info
data = data.join(df_personal_data)          # pridanie nových atribútov do nášho datasetu

Rozdelenie atribútu **class** do dvoch nových atribútov.

In [125]:
data['class'].head()

id
0    negative.|1231
1     negative.|329
2    negative.|2518
3     negative.|821
4     negative.|951
Name: class, dtype: object

In [126]:
#data = train_data
column1_name = 'class'
#Nepotrebujeme
column2_name = 'second_id'
df_class = pd.DataFrame(data["class"].str.split('\.\|').tolist(), columns = [column1_name,column2_name])
df_class['class'].value_counts()

negative               935
underreplacement        17
overreplacement         11
replacement therapy      9
Name: class, dtype: int64

In [127]:
# vymazanie atribútu class
data = data.drop(['class'], axis=1) 
# pridanie nových atribútov do nášho datasetu
data = data.join(df_class[column1_name])

In [128]:
data['class'].value_counts()

negative               935
underreplacement        17
overreplacement         11
replacement therapy      9
Name: class, dtype: int64

In [131]:
new_column = 'education_num'
#data['education-num']
data[new_column] = pd.to_numeric(data[new_column], errors='coerce')
data[new_column] = data[new_column].apply(lambda x: abs(x)/100 if abs(x) > 100 else x)
data[new_column].value_counts()

9.0     301
10.0    227
13.0    159
14.0     54
11.0     47
7.0      44
6.0      31
12.0     29
5.0      20
8.0      17
4.0      14
15.0     11
16.0      7
3.0       6
1.0       3
2.0       2
Name: education_num, dtype: int64

In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972 entries, 0 to 971
Data columns (total 43 columns):
age                          972 non-null int64
sex                          932 non-null float64
on_thyroxine                 972 non-null bool
query_on_thyroxine           972 non-null bool
on_antithyroid_medication    972 non-null bool
sick                         972 non-null bool
pregnant                     972 non-null bool
thyroid_surgery              972 non-null bool
I131_treatment               972 non-null bool
query_hypothyroid            972 non-null bool
query_hyperthyroid           972 non-null bool
lithium                      972 non-null bool
goitre                       972 non-null bool
tumor                        972 non-null bool
hypopituitary                972 non-null bool
psych                        972 non-null bool
TSH_measured                 972 non-null bool
TSH                          887 non-null float64
T3_measured                  972 non-null 

# Vychýlené hodnoty
V tejto časti sa snažíme spracovať vychýlené hodnoty numerických dát.

# Odvodenie chýbajúcich hodnôt po riadkoch

# Doplnenie chýbajúcich hodnôt po stĺpcoch

# Pomocné grafové examples

- data.fillna((0: 1, 1:1, 2:1))
- data.isnull().sum()
- data.dropna(how='all')
- data.plot.box()

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

ts = ts.cumsum()
    
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list('ABCD'))

df = df.cumsum()
df3 = pd.DataFrame(np.random.randn(1000, 2), columns=['B', 'C']).cumsum()

df3['A'] = pd.Series(list(range(len(df))))
df3.head()

In [ ]:
df.head(2)#.plot.hist(); plt.axhline(0, color='g')


In [ ]:
data.select_dtypes('int').plot.hist(alpha=0.5)

In [ ]:
df2 = pd.DataFrame(np.random.rand(10, 4), columns=['a', 'b', 'c', 'd'])

#df2.plot.bar(stacked=True);
plt.figure()
data['age'].plot.bar()


In [ ]:
df4 = pd.DataFrame({'a': np.random.randn(1000) + 1, 'b': np.random.randn(1000),'c': np.random.randn(1000) - 1}, columns=['a', 'b', 'c'])
plt.figure();

df4.plot.hist(alpha=0.5)

In [ ]:
df4.plot.hist(alpha=0.5, bins=200)

In [ ]:
plt.figure()
df['A'].head()

In [ ]:
df['A'].diff().head()

In [ ]:
df.diff().hist(color='k', alpha=0.5, bins=50)

In [ ]:
df = pd.DataFrame(np.random.rand(10, 5), columns=['A', 'B', 'C', 'D', 'E'])
color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
df.plot.box(color=color, sym='r+', positions=[1, 4, 5, 6, 8])

In [ ]:
df.boxplot()

In [ ]:
df = pd.DataFrame(np.random.rand(10,2), columns=['Col1', 'Col2'] )

df['X'] = pd.Series(['A','A','A','A','A','B','B','B','B','B'])

plt.figure();
df
#bp = df.boxplot(by='X')

In [ ]:
np.random.seed(1234)
df_box = pd.DataFrame(np.random.randn(50, 2))
df_box['g'] = np.random.choice(['A', 'B'], size=50)
df_box.loc[df_box['g'] == 'B', 1] += 3
bp = df_box.boxplot(by='g')

In [ ]:
df = pd.DataFrame(np.random.rand(50, 4), columns=['a', 'b', 'c', 'd'])

In [ ]:
ax = df.plot.scatter(x='a', y='b', color='DarkBlue', label='Group 1');

df.plot.scatter(x='c', y='d', color='DarkGreen', label='Group 2', ax=ax);

In [ ]:
df.plot.scatter(x='a', y='b', c='c', s=10);

# Zdroje
Príklady ako pracovať s pandas - https://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-plotting

Robenie grafov v pandas - https://pandas.pydata.org/pandas-docs/stable/visualization.html